In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
from collections import abc

import numpy as np

from harmonic_balance import freq
from harmonic_balance import arclength_continuation as alc

ndarray = np.ndarray

In [ ]:
NH = 4
n = 2
N = 18
m1, m2 = 1, 2
c1, c2, c3 = 1, 0.5, 2
k1, k2, k3 = 1, 0.5, 2

M = np.diag([m1, m2])
C = np.array([[c1 + c2, -c2], [-c2, c2 + c3]])
K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

ks = (1, 1)
dofs = (0, 1)
is_cosines = (1, 0)
coefficients = (2, 1)
b_ext = freq.get_b_ext(NH, n, ks, dofs, is_cosines, coefficients)


def get_slice(dof: int, N: int):
    return np.s_[dof * N : (dof + 1) * N]


def get_nonlinear(
    factor: float,
) -> tuple[
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
]:
    dof = 0
    s_ = lambda N: get_slice(dof, N)

    def f_nl(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * x[s_(N)] ** 3
        return res

    def df_nl_dx(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * 3 * x[s_(N)] ** 2
        return np.diag(res.ravel())

    def df_nl_d_xdot(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        shape = x.shape[0]
        return np.zeros((shape, shape), dtype=float)

    return f_nl, df_nl_dx, df_nl_d_xdot


f_nl, df_nl_dx, df_nl_d_xdot = get_nonlinear(0.1)

x = np.array([2, 3])
zeros = np.zeros_like(x)
f_nl(x, zeros, 1), df_nl_dx(x, zeros, 1), df_nl_d_xdot(x, zeros, 1)

(array([0.8, 0. ]),
 array([[1.2, 0. ],
        [0. , 0. ]]),
 array([[0., 0.],
        [0., 0.]]))

In [ ]:
z1, z0 = b_ext.toarray() + 0.01, b_ext.toarray()
omega1, omega0 = 2.01, 2.0
s = 0.1

(
    alc.get_P(z1, z0, omega1, omega0, s),
    alc.get_dP_dz(z1, z0),
    alc.get_dP_d_omega(omega1, omega0),
)

(np.float64(-0.008900000000000005),
 array([0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j]),
 0.019999999999999574)

In [ ]:
db_dz = alc.get_db_dz(omega0, z1, df_nl_dx, df_nl_d_xdot, NH, n, N)
db_dz.shape

(10, 10)

In [ ]:
A = freq.get_A(omega0, NH, M, C, K)
(
    alc.get_R(z1, omega0, A, f_nl, b_ext, NH, n, N).shape,
    alc.get_dR_dz(A, db_dz).shape,
    alc.get_dR_d_omega(z1, omega0, NH, M, C).shape,
)

((10,), (10, 10), (10,))